# Set up the sqlite DB 

TODO -- Take a stand on 1-dim vs multi-dim PKS
* Some of the code ALLOWS multi-dim
* Other parts only work with 1-dim, so it is misleading to pretend multi-dim is supported

In [4]:
import sys
from pathlib import Path

REPO_ROOT = Path("./").absolute().parent.parent
sys.path.append(str(REPO_ROOT))
DB_FILE = REPO_ROOT / "data/jamming.db"


In [2]:
### Init the DB
from jamdb.db import init_db

SQL_FILE = REPO_ROOT / "jamdb/jamming.sql"

DB_FILE.unlink(missing_ok=True)

init_db(sql_file=SQL_FILE, db_file=DB_FILE)

In [5]:
import json
import sqlite3
import pandas as pd
from pandas_ods_reader import read_ods

pd.options.display.max_rows = 200

from jamdb.db import BackendSQLite #, DBError, UniqueConstraintError, FKConstraintError

conn = BackendSQLite(DB_FILE)

In [ ]:
ODS_FILE = "../jamming_db/data/paul_k_db/public.ods"
ODS_FILE_PRIVATE = "../jamming_db/data/paul_k_db/private.ods"
table_metadata = read_ods(ODS_FILE, "table_metadata")
relations = read_ods(ODS_FILE, "relations")

for table_name in conn._sorted_tables():
    if table_name == "Person":
        pub = read_ods(ODS_FILE, "Person")
        priv = read_ods(ODS_FILE_PRIVATE, "PersonPrivate")
        table = pub.merge(priv, on="id")
    else:
        table = read_ods(ODS_FILE, table_name)

    for _, row in table.iterrows():
        try:
            conn.insert_row(table_name=table_name, row=row)
        except Exception as exc:
            msg = f"{table_name=}, {row=}"
            raise Exception(msg) from exc
    # try:
    #     table.to_sql(table_name, conn.conn, if_exists="append", index=False)
    #     conn.conn.commit()
    # except Exception as exc:
    #     raise Exception(table_name) from exc

In [ ]:
from jamdb.entities import Field

In [ ]:
# It is proving non-trivial to fetch various constraints from the db
# and some of if effectively comes back to parsing the `jamming.sql`
# Alternative, we could DEFINE all this stuff in a json, and then 
# create the `jamming.sql` and the DB itself from the json.

stuff = conn.query("select sql from sqlite_master where type='table' and name='EventGen'").iloc[0]["sql"]

# pattern = "unique"
# pattern = "not null"
pattern = "default"

entries = [
    tmp.strip() for tmp in stuff.splitlines()
    if pattern in tmp.lower()
]
entries

In [ ]:
relations = conn._get_relations()
entity = conn.entities["EventOcc"]
fields = {}

for column in entity.columns:
    # How to extract UNIQUE, REQUIRED, and DEFAULTs from sqllite metadata?
    unique = False
    required = False
    default = None
    
    relation = relations.query(f"left_table == '{entity.table_name}'").query(f"left_key == '{column}'")
    if len(relation) == 1:
        allowed_values = relation.iloc[0]["allowed_values"]
    else:
        allowed_values = None

    current_values = list(conn.query(f"SELECT DISTINCT {column} FROM {entity.table_name}")[column])

    field = Field(
        table_name=entity.table_name,
        field_name=column,
        required=required, 
        unique=unique,
        allowed_values=allowed_values,
        default=default,
        current_values=current_values
    )
    fields[column] = field

fields["event_gen_id"].allowed_values

In [ ]:
conn.query("SELECT * FROM _schema_columns")

In [ ]:
# table_names = conn._sorted_tables()
# idx = -1

idx +=1
table_name = table_names[idx]

print(table_name)
conn.query(f"SELECT * FROM {table_name}").head()

In [ ]:
conn.query("SELECT * FROM Key")["mode_id"].value_counts()

In [ ]:
conn.delete_row("Person", "paul_k")

In [ ]:
conn.delete_row("Song", "work_song")

In [ ]:
# conn.insert_row("PersonInstrument", {"id": "made_up", "person_id": "paul_k", "instrument_id": "gong"})

In [ ]:
conn.insert_row("PersonInstrument", {"id": "made_up", "person_id": "paul_k", "instrument_id": "trumpet"})

In [ ]:
conn.update_row("PersonInstrument", "made_up", {"id": "made_up", "instrument_id": "a_sax"})

In [ ]:
conn.update_row("PersonInstrument", "made_up", {"instrument_id": "e_guitar"})

In [ ]:
conn.update_row("PersonInstrument", "made_up", {"instrument_id": "gong"})

In [ ]:
conn.update_row("PersonInstrument", "made_up", {"instrument_id": "t_sax"})

In [ ]:
conn.get_row("PersonInstrument", "made_up")

In [ ]:
conn.rename_primary_key("PersonInstrument", "made_up", "paul_k:t_sax")

In [ ]:
conn.get_row("PersonInstrument", "paul_k:t_sax")

In [ ]:
conn.get_row("PersonInstrument", "paul_k:e_guitar")

In [ ]:
conn.rename_primary_key("PersonInstrument", "paul_k:e_guitar", "paul_k:electric_geetar")

In [ ]:
conn.get_row("PersonInstrument", "paul_k:electric_geetar")

In [ ]:
conn.get_row("PersonInstrument", "travis_p:a_sax")

In [ ]:
conn.delete_row("PersonInstrument", "travis_p:a_sax")

In [ ]:
# conn.query("SELECT * FROM PersonInstrument")
# conn.query("SELECT * FROM SongPerform")
# conn.query("SELECT * FROM EventGen")

In [ ]:
conn.conn.close()
conn._connect()

In [ ]:
conn.query("SELECT * FROM Key")

In [ ]:
conn.query("SELECT * FROM Song")["key_id"].drop_duplicates().sort_values()

In [ ]:
conn.query("SELECT * FROM SongPerform")["key_id"].drop_duplicates().sort_values()

In [ ]:
conn._connect()

In [ ]:
conn.get_row("PersonInstrument", "trav_p:alto_sax")

In [ ]:
conn.query("SELECT * FROM SongPerform")

In [ ]:
conn.query("SELECT * FROM Venue").head()

In [ ]:
print("\n".join(list(conn.entities.keys())))

In [ ]:
conn._create_erd()

In [ ]:
table = Table(conn, "PersonInstrument")

In [ ]:
conn.query(f"pragma foreign_key_list('{table_name}')")

In [ ]:
insert_person_inst({"id": "made_up", "person_id": "paul_k", "instrument_id": "gong"}, conn)

In [ ]:
table_name = "PersonInstrument"

stuff = conn.query(f"select sql from sqlite_master where type='table' and name='{table_name}'").iloc[0]["sql"]
print(stuff)

entries = [ tmp.strip() for tmp in stuff.splitlines() if tmp.lower().find("constraint")>=0 or tmp.lower().find("unique")>=0 ]
entries

In [ ]:
conn.execute(f"pragma index_list('{table_name}')").fetchall()

In [ ]:
conn.execute(f".schema {table_name}").fetchall()

In [ ]:
relations = []
for table_name in conn.table_names():
    relations.extend(
        {
            "left_table": table_name,
            "left_key": row["from"],
            "right_table": row["table"],
            "right_key": row["to"]
        }
        for _, row in conn.query(f"pragma foreign_key_list('{table_name}')").iterrows()
    )

for constraint in relations:
    constraint["allowed_values"] = list(
        conn.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
    )
relations

# for constraint in fks:
#     col_name = constraint["left_key"]
#     if constraint["left_key"] in row:
#         value = row[col_name]
        
#         constraint["allowed_values"] = list(
#             self.backend.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
#         )


# fks = (
#                 self.backend._get_relations().query(f'left_table == "{self.table_name}"')
#             ).to_dict(orient="records")
    
#             for constraint in fks:
#                 col_name = constraint["left_key"]
#                 if constraint["left_key"] in row:
#                     value = row[col_name]
                    
#                     constraint["allowed_values"] = list(
#                         self.backend.query(f'SELECT {constraint["right_key"]} FROM {constraint["right_table"]}')[constraint["right_key"]]
#                     )

In [ ]:
df = conn.query("SELECT table_name, column from _schema_columns")
df["column"] = df["column"].apply(lambda x: [x])
df.groupby("table_name")["column"].sum().to_dict()